In [52]:
import pandas as pd
import numpy as np


In [53]:
df = pd.read_csv('clean_data.csv', index_col=0)
df.head(3)

,WEEK OF,STATION,STATION CITY,STATION PROVINCE,STATION LATITUDE,STATION LONGITUDE,CHART POSITION,ARTIST NAME(S),ARTIST COUNTRY,ARTIST HOME CITY,...,LABEL TYPE,LANGUAGE OF MUSIC,VISIBLE ETHNIC MINORITY,CENSUS RACE CLASSIFICATION,ARTIST GENDER,M,A,P,L,CANADIAN CONTENT
0,2006-01-10,cjsr,edmonton,ab,53.55,-113.5,4,breakestra,us,"los angeles, ca",...,indie,english,yes,mixed group,male group,no,no,no,no,no
1,2006-01-10,cjsr,edmonton,ab,53.55,-113.5,5,candy's .22,us,"los angeles, ca",...,indie,english,no,white,male group,no,no,no,no,no
2,2006-01-10,cjsr,edmonton,ab,53.55,-113.5,7,dangerdoom,us,"new york, ny",...,indie,english,yes,black,male,no,no,no,no,no


In [77]:
def expand_column_to_percentiles(column:str, group:str, dataframe:pd.DataFrame, origin:pd.DataFrame):
    for x in sorted(df[column].unique()):
        x_plays = origin[df[column]==x ].groupby(group)[column].count()
        total_plays = origin.groupby(group)[column].count()
        if type(x) == np.float64:
            x = int(x)
        value = (x_plays / total_plays)
        if np.isnan(value).any():
            value = 0
        dataframe[f'{column} {x}'] = value

In [78]:
artist_data = df.groupby('ARTIST NAME(S)').agg(lambda x: pd.Series.mode(x, dropna=False)[0])[
                                                [ 'ARTIST HOME LATITUDE',
                                                  'ARTIST HOME LONGITUDE',
                                                  'ARTIST HOME CITY',
                                                  'ARTIST COUNTRY',
                                                  'LANGUAGE OF MUSIC',
                                                  'VISIBLE ETHNIC MINORITY',
                                                  'CENSUS RACE CLASSIFICATION',
                                                  'ARTIST GENDER',
                                                  'CANADIAN CONTENT']
                                                ]
artist_data['ALBUMS PLAYED'] =  df.groupby('ARTIST NAME(S)')['ALBUM NAME'].nunique()
artist_data['STATIONS PLAYED ON'] =  df.groupby('ARTIST NAME(S)')['STATION'].nunique()
artist_data['MEAN CHART POSITION'] =  df.groupby('ARTIST NAME(S)')['CHART POSITION'].mean().round().astype(int)
artist_data['TOTAL DAYS PLAYED'] =  df.groupby('ARTIST NAME(S)')['ALBUM NAME'].count()

expand_column_to_percentiles('STATION', 'ARTIST NAME(S)', artist_data, df)
expand_column_to_percentiles('CHART POSITION', 'ARTIST NAME(S)', artist_data, df)


In [81]:
artist_data.reset_index().to_csv('artist_table.csv')

In [83]:
station_data = df.groupby('STATION').agg(lambda x: pd.Series.mode(x, dropna=False)[0])[
                                                [ 'STATION LATITUDE',
                                                  'STATION LONGITUDE',
                                                  'STATION CITY',
                                                  'STATION PROVINCE',
                                                  'ARTIST HOME CITY',
                                                  'ARTIST COUNTRY',
                                                  'LABEL TYPE',
                                                  'LABEL NAME'
                                                  ]
                                                ]

expand_column_to_percentiles('ARTIST GENDER', 'STATION', station_data, df)
expand_column_to_percentiles('CENSUS RACE CLASSIFICATION', 'STATION', station_data, df)
expand_column_to_percentiles('ARTIST COUNTRY', 'STATION', station_data, df)
expand_column_to_percentiles('CANADIAN CONTENT', 'STATION', station_data, df)
station_data.rename(columns={ 'ARTIST HOME CITY' : 'MOST PLAYED ARTIST CITY',
                              'LABEL TYPE' : 'MOST PLAYED LABEL TYPE',
                              'LABEL NAME' : 'MOST PLAYED LABEL ',
                              'yes' : 'CAN CON', 
                              'no' : 'NOT CAN CON'

                             }, inplace=True)



In [84]:
station_data.reset_index().to_csv('station_table.csv')

In [85]:
spider_map_data = pd.DataFrame(columns=['STATION-ARTIST', 'STATION', 'ARTIST', 'PATH ID', 'LATITUDE','LONGITUDE', 'WEEK OF'])

In [86]:
weekly_plays = df.groupby(['WEEK OF','STATION','ARTIST NAME(S)'])['ARTIST NAME(S)'].count().reset_index(name='count')
weekly_plays['count'].value_counts()

1    35236
2      273
3       24
4        1
Name: count, dtype: int64

In [87]:
df['WEEK OF'].unique()

array(['2006-01-10', '2006-01-17', '2006-01-24', '2006-01-31',
       '2006-02-07', '2006-02-14', '2006-02-21', '2006-02-28',
       '2006-03-07', '2006-03-14', '2006-03-21', '2006-03-28',
       '2006-04-04', '2006-04-11', '2006-04-18', '2006-04-25',
       '2006-05-02', '2006-05-09', '2006-05-16', '2006-05-23',
       '2006-05-30', '2006-06-06', '2006-06-13', '2006-06-20',
       '2006-06-27', '2006-07-04', '2006-07-11', '2006-07-18',
       '2006-07-25', '2006-08-01', '2006-08-08', '2006-08-15',
       '2006-08-22', '2006-08-29', '2006-09-05', '2006-09-12',
       '2006-09-19', '2006-09-26', '2006-10-03', '2006-10-10',
       '2006-10-17', '2006-10-24', '2006-10-31', '2006-11-07',
       '2006-11-14', '2006-11-21', '2006-11-28', '2006-12-05',
       '2006-12-12', '2006-12-19', '2006-12-26', '2007-01-02',
       '2007-01-09', '2007-01-16', '2007-01-23', '2007-01-30',
       '2007-02-06', '2007-02-13', '2007-02-20', '2007-02-27',
       '2007-03-06', '2007-03-13', '2007-03-20', '2007-

In [88]:
df[df['WEEK OF'] == '2006-07-04']

,WEEK OF,STATION,STATION CITY,STATION PROVINCE,STATION LATITUDE,STATION LONGITUDE,CHART POSITION,ARTIST NAME(S),ARTIST COUNTRY,ARTIST HOME CITY,...,LABEL TYPE,LANGUAGE OF MUSIC,VISIBLE ETHNIC MINORITY,CENSUS RACE CLASSIFICATION,ARTIST GENDER,M,A,P,L,CANADIAN CONTENT
3590,2006-07-04,cjsw,calgary,ab,51.08,-114.08,1,dabrye,us,"ann arbor, mi",...,indie,english,no,white,male,no,no,no,no,no
3591,2006-07-04,cjsw,calgary,ab,51.08,-114.08,2,aloe blacc,us,"orange county, ca",...,indie,english,yes,black,male,no,no,no,no,no
3592,2006-07-04,cjsw,calgary,ab,51.08,-114.08,4,mr. lif,us,"boston, mass",...,indie,english,yes,black,male,no,no,no,no,no
3593,2006-07-04,cjsw,calgary,ab,51.08,-114.08,5,krs-one,us,"new york, ny",...,indie,english,yes,black,male,no,no,no,no,no
3594,2006-07-04,cjsw,calgary,ab,51.08,-114.08,7,kenny dope,us,"brooklyn, ny",...,indie,english,yes,black,male,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,2006-07-04,ckut,montreal,qc,45.50,-73.58,6,mathematics,us,"new york, ny",...,indie,english,yes,black,male,no,no,no,no,no
3676,2006-07-04,ckut,montreal,qc,45.50,-73.58,7,aloe blacc,us,"orange county, ca",...,indie,english,yes,black,male,no,no,no,no,no
3677,2006-07-04,ckut,montreal,qc,45.50,-73.58,8,dj spinna,us,"new york, ny",...,indie,english,yes,black,male,no,no,no,no,no
3678,2006-07-04,ckut,montreal,qc,45.50,-73.58,9,mr. lif,us,"boston, mass",...,indie,english,yes,black,male,no,no,no,no,no


In [89]:
df[df['WEEK OF'] == '2006-07-04']['STATION'].unique()

array(['cjsw', 'ckxu', 'cfbx', 'cjum', 'chrw', 'chry', 'cilu', 'cjam',
       'ckut'], dtype=object)

In [93]:
df[(df['WEEK OF'] == '2006-07-04') & (df['STATION'] == 'cfbx')]['ARTIST NAME(S)'].unique()

array(['giant panda', 'soul position', 'perseverence', 'ice cube',
       'aloe blacc', 'cedille', 'reverse engineering', 'fax4', 'phatt al',
       'unknown'], dtype=object)

In [102]:
df[(df['WEEK OF'] == '2006-07-04') & (df['STATION'] == 'cfbx') ]['ARTIST NAME(S)'].value_counts()['giant panda']

1

In [ ]:
spider_map_data = pd.DataFrame(columns=['STATION-ARTIST', 'STATION', 'ARTIST', 'PATH ID', 'LATITUDE','LONGITUDE', 'WEEK OF', 'PLAYS'])

In [108]:
def build_spider_map_dataframe():
    rows = []
    
    for date in df['WEEK OF'].unique():
        for station in df[df['WEEK OF'] == date]['STATION'].unique():
            for artist in df[(df['WEEK OF'] == date) & (df['STATION'] == station)]['ARTIST NAME(S)'].unique():
                
                station = station
                station_lat = station_data.loc[station]['STATION LATITUDE']
                station_long = station_data.loc[station]['STATION LONGITUDE']
                artist = artist
                artist_lat = artist_data.loc[artist]['ARTIST HOME LATITUDE']
                artist_long = artist_data.loc[artist]['ARTIST HOME LONGITUDE']
                week = date
                plays = df[(df['WEEK OF'] == date) & (df['STATION'] == station)]['ARTIST NAME(S)'].value_counts()[artist]
                origin = {'STATION-ARTIST':'station', 
                        'STATION': station, 
                        'ARTIST': artist, 
                        'PATH ID':f'{station}-{artist}', 
                        'LATITUDE':station_lat,
                        'LONGITUDE':station_long, 
                        'WEEK OF':week, 
                        'PLAYS': 0}
                
                destination = {'STATION-ARTIST':'artist', 
                        'STATION': station, 
                        'ARTIST': artist, 
                        'PATH ID':f'{station}-{artist}', 
                        'LATITUDE':artist_lat,
                        'LONGITUDE':artist_long, 
                        'WEEK OF':week, 
                        'PLAYS' : plays}
                if not np.isnan([artist_lat,artist_long]).any():
                    rows.append(origin)
                    rows.append(destination)
        
    return pd.DataFrame(rows)
        

In [109]:
spider_map_data  = build_spider_map_dataframe()


spider_map_data.to_csv('spider_map.csv')

In [115]:
spider_map_data

,STATION-ARTIST,STATION,ARTIST,PATH ID,LATITUDE,LONGITUDE,WEEK OF,PLAYS
0,station,cjsr,breakestra,cjsr-breakestra,53.55,-113.50,2006-01-10,0
1,artist,cjsr,breakestra,cjsr-breakestra,34.05,-118.24,2006-01-10,1
2,station,cjsr,candy's .22,cjsr-candy's .22,53.55,-113.50,2006-01-10,0
3,artist,cjsr,candy's .22,cjsr-candy's .22,34.05,-118.24,2006-01-10,1
4,station,cjsr,dangerdoom,cjsr-dangerdoom,53.55,-113.50,2006-01-10,0
...,...,...,...,...,...,...,...,...
68089,artist,ckut,swollen members,ckut-swollen members,49.25,-123.13,2010-03-02,1
68090,station,ckut,chokeules,ckut-chokeules,45.50,-73.58,2010-03-02,0
68091,artist,ckut,chokeules,ckut-chokeules,43.66,-79.42,2010-03-02,1
68092,station,ckut,radio radio,ckut-radio radio,45.50,-73.58,2010-03-02,0


In [114]:
spider_map_data['PLAYS'].value_counts()

0    34047
1    34008
2       39
Name: PLAYS, dtype: int64